In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/stat940project')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

%reload_ext autoreload

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np
import random
import math
import torch.optim as optim
from tqdm.auto import tqdm, trange

In [ ]:
from dataset import load_data, save_data, load_targets, AdversarialDataset, load_adv_data
from utils.augmentations import *
from utils.utils import run, set_seed, pcshow
from models import Pointnet, DGCNN
from attacks import FGM, IFGM, PGD

In [ ]:
set_seed(1)

In [ ]:
parent_dir = '/content/drive/MyDrive/stat940project'
data_path = os.path.join(parent_dir, 'data/ModelNet40')

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')

else:
    device = torch.device('cpu')
    print('running on CPU')

test_X, test_y = load_data(data_path, mode='test')
test_targets = load_targets(data_path)



running on GPU


In [ ]:
default_transforms = transforms.Compose(
    [
        ToTensor()
    ]
)

In [ ]:
testset = AdversarialDataset(test_X, test_y, test_targets, transforms=default_transforms)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
def run(targeted, file_name, output_path, model_name, signed = False):
      if model_name == "pointnet":
          model = Pointnet(device=device).to(device)
      else:
          model = DGCNN().to(device)
      checkpoint_path = os.path.join(parent_dir, f'checkpoints/{model_name}/best.pt')


      model.load_state_dict(torch.load(checkpoint_path, map_location=device)["model_state_dict"])
      model.eval()

      all_adv_pc = []
      all_gt_labels = []
      all_target_labels = []
      all_success = 0

      attacker = PGD(model, epsilon=0.2)
      for pc, labels, targets in tqdm(testloader):
          with torch.no_grad():
              pc = pc.to(device)
              labels = labels.to(device)
              targets = targets.to(device)

          if targeted:
              adv_pc, success_num = attacker.attack(pc, targets, device, targeted=targeted)
          else:
              adv_pc, success_num = attacker.attack(pc, labels, device, targeted=targeted)

          all_adv_pc += [adv_pc]
          all_gt_labels += [labels.detach().cpu().numpy()]
          all_target_labels += [targets.detach().cpu().numpy()]
          all_success += success_num
          print()

      print(f"Success Rate: {all_success / len(test_X)}")


      all_adv_pc = np.concatenate(all_adv_pc, axis=0)
      all_gt_labels = np.concatenate(all_gt_labels, axis=0)
      all_target_labels = np.concatenate(all_target_labels, axis=0)



      save_data(all_adv_pc, file_name, output_path)
      save_data(all_gt_labels, 'test_labels', output_path)
      save_data(all_adv_pc, 'target_labels', output_path)
      return all_adv_pc

In [ ]:
path = os.path.join(parent_dir, 'data/PGD/Pointnet')
if not os.path.exists(path):
    os.makedirs(path)
all_adv_pc = run(False, 'adv_test_untarget', path, 'pointnet', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

14/64 successful attacks in iteration  0
20/64 successful attacks in iteration  1
25/64 successful attacks in iteration  2
29/64 successful attacks in iteration  3
32/64 successful attacks in iteration  4
35/64 successful attacks in iteration  5
38/64 successful attacks in iteration  6
40/64 successful attacks in iteration  7
46/64 successful attacks in iteration  8
48/64 successful attacks in iteration  9
48/64 Total successful attacks

13/64 successful attacks in iteration  0
15/64 successful attacks in iteration  1
19/64 successful attacks in iteration  2
23/64 successful attacks in iteration  3
28/64 successful attacks in iteration  4
30/64 successful attacks in iteration  5
31/64 successful attacks in iteration  6
36/64 successful attacks in iteration  7
39/64 successful attacks in iteration  8
40/64 successful attacks in iteration  9
42/64 Total successful attacks

13/64 successful attacks in iteration  0
14/64 successful attacks in iteration  1
21/64 successful attacks in iterat

In [ ]:
x1, y1, z1 = all_adv_pc[1].T
x2, y2, z2 = test_X[1].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='blue')

In [ ]:
path = os.path.join(parent_dir, 'data/PGD/Pointnet')
all_adv_pc = run(True, 'adv_test_target', path, 'pointnet', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

0/64 successful attacks in iteration  0
0/64 successful attacks in iteration  1
0/64 successful attacks in iteration  2
0/64 successful attacks in iteration  3
0/64 successful attacks in iteration  4
1/64 successful attacks in iteration  5
2/64 successful attacks in iteration  6
2/64 successful attacks in iteration  7
2/64 successful attacks in iteration  8
3/64 successful attacks in iteration  9
6/64 Total successful attacks

0/64 successful attacks in iteration  0
0/64 successful attacks in iteration  1
0/64 successful attacks in iteration  2
0/64 successful attacks in iteration  3
0/64 successful attacks in iteration  4
0/64 successful attacks in iteration  5
1/64 successful attacks in iteration  6
1/64 successful attacks in iteration  7
2/64 successful attacks in iteration  8
2/64 successful attacks in iteration  9
2/64 Total successful attacks

1/64 successful attacks in iteration  0
1/64 successful attacks in iteration  1
1/64 successful attacks in iteration  2
1/64 successful at

In [ ]:
x1, y1, z1 = all_adv_pc[2].T
x2, y2, z2 = test_X[2].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='blue')

In [ ]:
path = os.path.join(parent_dir, 'data/PGD/DGCNN')
if not os.path.exists(path):
    os.makedirs(path)
all_adv_pc = run(False, 'adv_test_untarget', path, 'dgcnn', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

5/64 successful attacks in iteration  0
7/64 successful attacks in iteration  1
11/64 successful attacks in iteration  2
11/64 successful attacks in iteration  3
14/64 successful attacks in iteration  4
14/64 successful attacks in iteration  5
16/64 successful attacks in iteration  6
17/64 successful attacks in iteration  7
18/64 successful attacks in iteration  8
20/64 successful attacks in iteration  9
24/64 Total successful attacks

4/64 successful attacks in iteration  0
6/64 successful attacks in iteration  1
11/64 successful attacks in iteration  2
11/64 successful attacks in iteration  3
13/64 successful attacks in iteration  4
13/64 successful attacks in iteration  5
16/64 successful attacks in iteration  6
16/64 successful attacks in iteration  7
20/64 successful attacks in iteration  8
20/64 successful attacks in iteration  9
22/64 Total successful attacks

8/64 successful attacks in iteration  0
8/64 successful attacks in iteration  1
10/64 successful attacks in iteration  2

In [ ]:
x1, y1, z1 = all_adv_pc[6].T
x2, y2, z2 = test_X[6].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='blue')

In [ ]:
path = os.path.join(parent_dir, 'data/PGD/DGCNN')
all_adv_pc = run(True, 'adv_test_target', path, 'dgcnn', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

0/64 successful attacks in iteration  0
0/64 successful attacks in iteration  1
0/64 successful attacks in iteration  2
0/64 successful attacks in iteration  3
0/64 successful attacks in iteration  4
0/64 successful attacks in iteration  5
0/64 successful attacks in iteration  6
0/64 successful attacks in iteration  7
0/64 successful attacks in iteration  8
2/64 successful attacks in iteration  9
2/64 Total successful attacks

0/64 successful attacks in iteration  0
0/64 successful attacks in iteration  1
0/64 successful attacks in iteration  2
0/64 successful attacks in iteration  3
0/64 successful attacks in iteration  4
0/64 successful attacks in iteration  5
0/64 successful attacks in iteration  6
0/64 successful attacks in iteration  7
0/64 successful attacks in iteration  8
0/64 successful attacks in iteration  9
0/64 Total successful attacks

0/64 successful attacks in iteration  0
0/64 successful attacks in iteration  1
0/64 successful attacks in iteration  2
0/64 successful at

In [ ]:
x1, y1, z1 = all_adv_pc[4].T
x2, y2, z2 = test_X[4].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='blue')